In [ ]:
import pandas as pd
from Bio import Entrez
from Bio import Medline
from collections import defaultdict
from datetime import datetime
import re
import http.client  # for catching IncompleteRead
from http.client import IncompleteRead

def parse_author_name(author_name):
    """
    Given an author name like 'Smith, John', returns (last, initials).
    Or if the name is 'John Smith', also returns (last, initials).
    """
    if ',' in author_name:
        last, first = author_name.split(',', 1)
        last = last.strip()
        first = first.strip()
        initials = ''.join([part[0] for part in first.split() if part])
    else:
        parts = author_name.split()
        if len(parts) == 1:
            last = parts[0]
            initials = ""
        else:
            last = parts[-1]
            first_parts = parts[:-1]
            initials = ''.join([p[0] for p in first_parts])
    return last, initials

def match_author(record_author, target_last, target_initials):
    parts = record_author.split()
    if len(parts) < 2:
        return False
    last_name = parts[0]
    initials = "".join(parts[1:])
    return (
        last_name.lower() == target_last.lower()
        and initials.lower().startswith(target_initials.lower())
    )

def get_pubmed_ids_for_author(author_name, email):
    """
    Return a list of PubMed IDs for a given author name.
    """
    Entrez.email = email or "your_email@example.com"
    
    target_last, target_initials = parse_author_name(author_name)
    if target_initials:
        query = f'{target_last} {target_initials}[Author]'
    else:
        query = f'{target_last}[Author]'

    handle = Entrez.esearch(db="pubmed", term=query, retmax=100000)
    record = Entrez.read(handle)
    handle.close()
    pmid_list = record.get("IdList", [])
    return pmid_list, target_last, target_initials

def get_author_positions_by_year(pmid_list, target_last, target_initials, email, hire_date=None):
    """
    For each PubMed ID, fetch the MEDLINE record and determine whether the target author is
    first, middle, or last. Only include papers published AFTER the given hire_date (if provided).
    Return a dict of { year : {"first": count, "middle": count, "last": count} }.
    """
    Entrez.email = email or "your_email@example.com"
    
    year_data = defaultdict(lambda: {"first": 0, "middle": 0, "last": 0})
    BATCH_SIZE = 200

    for start in range(0, len(pmid_list), BATCH_SIZE):
        batch = pmid_list[start:start+BATCH_SIZE]
        try:
            handle = Entrez.efetch(db="pubmed", id=batch, rettype="medline", retmode="text")
            records = Medline.parse(handle)

            for record in records:
                if 'AU' not in record:
                    continue
                authors = record['AU']
                
                # Attempt to parse the publication date
                dp = record.get('DP', '')  # e.g. '2022 Dec', '2022', '2022 Jul 1'
                pub_date = None
                try:
                    # Try to parse the entire date with pandas
                    pub_date = pd.to_datetime(dp, errors='coerce')
                except:
                    pass

                # If parsing failed or returned NaT, fallback to year-based approach
                if pub_date is None or pd.isnull(pub_date):
                    # If dp is at least 4 chars and the first 4 are digits, treat that as the year
                    if len(dp) >= 4 and dp[:4].isdigit():
                        try:
                            year_only = int(dp[:4])
                            # Approximate it to Jan 1 of that year
                            pub_date = datetime(year_only, 1, 1)
                        except:
                            # Could not parse year, skip
                            continue
                    else:
                        # Could not parse DP in any useful way
                        continue

                # If we have a hire_date, skip publications published on or before the hire_date
                if hire_date is not None and pub_date <= hire_date:
                    continue

                pub_year = str(pub_date.year)  # We’ll store the counts under the year

                # Find all positions of the target author in the author list
                found_positions = [
                    i for i, a in enumerate(authors) 
                    if match_author(a, target_last, target_initials)
                ]
                for pos in found_positions:
                    if pos == 0:
                        year_data[pub_year]["first"] += 1
                    elif pos == len(authors) - 1:
                        year_data[pub_year]["last"] += 1
                    else:
                        year_data[pub_year]["middle"] += 1
        finally:
            # Always close the handle, even if there's an exception
            handle.close()

    return year_data

def get_year_metrics(author_name, email, hire_date=None):
    """
    Wrapper function to return a dictionary of year-based metrics
    {year: {"first", "middle", "last", "total", "first_fraction", "last_fraction"}}
    Only includes papers after the given hire_date if provided.
    """
    pmid_list, target_last, target_initials = get_pubmed_ids_for_author(author_name, email)
    year_data = get_author_positions_by_year(
        pmid_list, target_last, target_initials, email, hire_date=hire_date
    )
    
    results = {}
    for year, data in year_data.items():
        total = data["first"] + data["middle"] + data["last"]
        first_fraction = round(data['first']/total, 2) if total else 0
        last_fraction = round(data['last']/total, 2) if total else 0
        results[year] = {
            "first": data["first"],
            "middle": data["middle"],
            "last": data["last"],
            "total": total,
            "first_fraction": first_fraction,
            "last_fraction": last_fraction
        }
    return results

def main():
    input_file = "input.xlsx"   # Change to your actual input file path
    output_file = "output.xlsx" # Change to desired output path
    
    df = pd.read_excel(input_file)
    
    all_years = set()
    result_rows = []
    skipped_authors = []
    processed_authors = []

    for idx, row in df.iterrows():
        raw_name = str(row["Name"])
        
        # Remove the (xxxxxx) part if present
        name_cleaned = re.sub(r"\(\d+\)", "", raw_name).strip()
        
        print(f"Processing author: {name_cleaned}")
        
        email = row.get("Email", "your_email@example.com")
        
        # ---- Parse the hire date (if available) into a datetime ----
        hire_date_value = row.get("Hire Date", None)
        if pd.notnull(hire_date_value):
            # Attempt to convert to datetime
            hire_date = pd.to_datetime(hire_date_value, errors='coerce')
        else:
            hire_date = None

        # Try/Except block to catch errors (like IncompleteRead) and skip problematic authors
        try:
            # Retrieve the publication metrics (only after hire_date)
            metrics_dict = get_year_metrics(name_cleaned, email, hire_date=hire_date)

            # Build a dictionary that stores the row's original data + new year columns
            row_data = row.to_dict()
            
            for year, info in metrics_dict.items():
                row_data[f"{year}-first"] = info["first"]
                row_data[f"{year}-middle"] = info["middle"]
                row_data[f"{year}-last"] = info["last"]
                row_data[f"{year}-fraction_first"] = info["first_fraction"]
                row_data[f"{year}-fraction_last"] = info["last_fraction"]
                row_data[f"{year}-total"] = info["total"]
                all_years.add(year)
            
            result_rows.append(row_data)
            processed_authors.append(name_cleaned)

        except Exception as e:
            print(f"Error processing author '{name_cleaned}': {e}")
            print("Skipping this author...")
            skipped_authors.append(name_cleaned)
            # Continue to the next author
            continue
    
    new_df = pd.DataFrame(result_rows)
    new_df.to_excel(output_file, index=False)
    print(f"\nOutput written to {output_file}")

    # Print a final summary
    print("\nFinal Summary:")
    print(f"Processed authors: {processed_authors}")
    print(f"Skipped authors due to errors: {skipped_authors}")

if __name__ == "__main__":
    main()


Processing author: Claudio Brunstein
Processing author: Alireza Navadeh
Processing author: Anita Reddy
Processing author: Kamonpun Ussavarungsi
Processing author: Neha Solanki
Processing author: Matthew Dettmer
Processing author: Emily Pennington
Processing author: Sudhir Krishnan
Processing author: Matthew Siuba
Processing author: Umur Hatipoglu
Processing author: Wayne Tsuang
Processing author: Jorge Ataucuri-Vargas
Processing author: Loutfi Aboussouan
Processing author: Peter Mazzone
Processing author: Francisco Almeida
Processing author: Sameep Sehgal
Processing author: Margaret Kuder
Processing author: Bo Xu
Processing author: Jona Banzon
Processing author: James Fernandez
Processing author: Donald Dumford III
Processing author: Nabin Shrestha
Processing author: Ken Wong
Processing author: Eduardo Mireles-Cabodevila
Processing author: Deepakraj Gajanana
Processing author: Stephen Ellis
Processing author: Ajit Moghekar
Processing author: Oussama Wazni
Processing author: Jessica Lov